# N8N Admin Dashboard - Workflow Control Panel Verification

This notebook validates the functionality of the N8N Admin Dashboard's Workflow Control Panel.

## Test Sequence:
1. Authentication Setup and Verification
2. System Status Check
3. Workflow Control Interface Testing
4. Integration Testing
5. Status Report Generation

## 1. Authentication Setup

First, let's import required libraries and set up the authentication configuration:

In [ ]:
# Import required libraries
import os
import requests
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import json

# N8N API Configuration
N8N_URL = 'https://n8n-bulletproof-v2.fly.dev'  # Default URL
N8N_API_KEY = ''  # Will be set through configuration

# Create input widgets for configuration
url_input = widgets.Text(
    value=N8N_URL,
    description='N8N URL:',
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)

api_key_input = widgets.Password(
    value=N8N_API_KEY,
    description='API Key:',
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)

status_output = widgets.Output()

def update_config(button):
    """Update configuration and verify connection"""
    global N8N_URL, N8N_API_KEY
    with status_output:
        clear_output(wait=True)
        N8N_URL = url_input.value
        N8N_API_KEY = api_key_input.value
        print("🔄 Updating configuration...")
        
        # Test connection
        headers = {'X-N8N-API-KEY': N8N_API_KEY}
        try:
            # Test health endpoint
            health_response = requests.get(f"{N8N_URL}/healthz", timeout=5)
            if health_response.status_code != 200:
                print(f"❌ Health check failed: {health_response.status_code}")
                return
            
            # Test API access
            api_response = requests.get(f"{N8N_URL}/api/v1/workflows", headers=headers, timeout=5)
            if api_response.status_code == 200:
                print("✅ Configuration updated successfully!")
                print("✅ API connection verified!")
            else:
                print(f"❌ API check failed: {api_response.status_code}")
        except Exception as e:
            print(f"❌ Connection error: {str(e)}")

update_button = widgets.Button(
    description='Update & Verify',
    button_style='primary'
)
update_button.on_click(update_config)

# Display configuration interface
print("Enter N8N URL and API Key to begin testing:")
display(widgets.VBox([
    url_input,
    api_key_input,
    update_button,
    status_output
]))

## 2. System Status Verification

Let's verify system connectivity and API access:

In [ ]:
def check_system_status():
    """Comprehensive system status check"""
    status_data = []
    headers = {'X-N8N-API-KEY': N8N_API_KEY}
    
    # Check N8N health
    try:
        health_response = requests.get(f"{N8N_URL}/healthz", timeout=5)
        status_data.append({
            'Component': 'N8N Service',
            'Status': '🟢 Online' if health_response.status_code == 200 else '🔴 Offline',
            'Message': f'Response: {health_response.status_code}'
        })
    except Exception as e:
        status_data.append({
            'Component': 'N8N Service',
            'Status': '🔴 Offline',
            'Message': str(e)
        })
    
    # Check API access
    try:
        api_response = requests.get(f"{N8N_URL}/api/v1/workflows", headers=headers, timeout=5)
        status_data.append({
            'Component': 'API Access',
            'Status': '🟢 Authorized' if api_response.status_code == 200 else '🔴 Unauthorized',
            'Message': f'Response: {api_response.status_code}'
        })
    except Exception as e:
        status_data.append({
            'Component': 'API Access',
            'Status': '🔴 Error',
            'Message': str(e)
        })
    
    # Display status
    status_df = pd.DataFrame(status_data)
    display(status_df)
    
    # Return True if all checks pass
    return all(row['Status'].startswith('🟢') for _, row in status_df.iterrows())

# Run system status check
system_ready = check_system_status()

## 3. Workflow Control Interface

Now let's implement and test the workflow control panel:

In [ ]:
# Create workflow control widgets
workflow_dropdown = widgets.Dropdown(
    description='Workflow:',
    style={'description_width': 'initial'},
    layout={'width': '60%'}
)

status_label = widgets.HTML(
    value='<span style="color: gray;">No workflow selected</span>'
)

toggle_button = widgets.ToggleButton(
    value=False,
    description='Inactive',
    button_style='danger',
    tooltip='Toggle workflow active status',
    icon='power-off',
    layout={'width': '150px'}
)

control_output = widgets.Output()

def get_workflows():
    """Fetch all workflows from N8N"""
    try:
        headers = {'X-N8N-API-KEY': N8N_API_KEY}
        response = requests.get(f"{N8N_URL}/api/v1/workflows", headers=headers)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error fetching workflows: {str(e)}")
        return []

def update_workflow_status(workflow_id, active):
    """Update workflow active status"""
    try:
        headers = {'X-N8N-API-KEY': N8N_API_KEY}
        endpoint = f"{N8N_URL}/api/v1/workflows/{workflow_id}/{'activate' if active else 'deactivate'}"
        response = requests.post(endpoint, headers=headers)
        response.raise_for_status()
        return True
    except Exception as e:
        print(f"Error updating workflow status: {str(e)}")
        return False

def refresh_workflows(*_):
    """Update workflow dropdown list"""
    workflows = get_workflows()
    workflow_dropdown.options = [(f"{w['name']} (ID: {w['id']})", w['id']) for w in workflows]
    workflow_dropdown.value = None
    toggle_button.value = False
    status_label.value = '<span style="color: gray;">No workflow selected</span>'

def on_workflow_select(change):
    """Handle workflow selection"""
    if change['new'] is None:
        toggle_button.value = False
        status_label.value = '<span style="color: gray;">No workflow selected</span>'
        return
    
    workflows = get_workflows()
    selected = next((w for w in workflows if w['id'] == change['new']), None)
    if selected:
        toggle_button.value = selected.get('active', False)
        toggle_button.description = 'Active' if toggle_button.value else 'Inactive'
        toggle_button.button_style = 'success' if toggle_button.value else 'danger'
        status = 'active' if toggle_button.value else 'inactive'
        status_label.value = f'<span style="color: {"green" if toggle_button.value else "red"}">Status: {status}</span>'

def on_toggle(change):
    """Handle workflow activation/deactivation"""
    if workflow_dropdown.value is None:
        return
    
    with control_output:
        clear_output(wait=True)
        print(f"{'Activating' if change['new'] else 'Deactivating'} workflow...")
        success = update_workflow_status(workflow_dropdown.value, change['new'])
        
        if success:
            toggle_button.description = 'Active' if change['new'] else 'Inactive'
            toggle_button.button_style = 'success' if change['new'] else 'danger'
            status = 'active' if change['new'] else 'inactive'
            status_label.value = f'<span style="color: {"green" if change["new"] else "red"}">Status: {status}</span>'
            print(f"✅ Workflow successfully {status}")
        else:
            toggle_button.value = not change['new']  # Revert on failure
            print("❌ Failed to update workflow status")

# Wire up event handlers
workflow_dropdown.observe(on_workflow_select, names='value')
toggle_button.observe(on_toggle, names='value')

# Create refresh button
refresh_button = widgets.Button(
    description='Refresh List',
    icon='sync',
    tooltip='Refresh workflow list',
    layout={'width': '150px'}
)
refresh_button.on_click(refresh_workflows)

# Create control panel layout
controls = widgets.VBox([
    widgets.HBox([workflow_dropdown, refresh_button], layout=widgets.Layout(align_items='center')),
    widgets.HBox([toggle_button, status_label], layout=widgets.Layout(align_items='center', padding='10px 0')),
    control_output
])

# Display control panel
print("Workflow Control Panel:")
display(controls)

# Initial workflow list population
refresh_workflows()

## 4. Test Report Generation

Generate a comprehensive test report of all operations:

In [ ]:
def generate_test_report():
    """Generate comprehensive test report"""
    report_data = []
    headers = {'X-N8N-API-KEY': N8N_API_KEY}
    
    # Test System Connectivity
    try:
        health_response = requests.get(f"{N8N_URL}/healthz", timeout=5)
        report_data.append({
            'Test': 'System Health Check',
            'Status': '✅ Passed' if health_response.status_code == 200 else '❌ Failed',
            'Details': f'Response: {health_response.status_code}'
        })
    except Exception as e:
        report_data.append({
            'Test': 'System Health Check',
            'Status': '❌ Failed',
            'Details': str(e)
        })
    
    # Test API Authorization
    try:
        api_response = requests.get(f"{N8N_URL}/api/v1/workflows", headers=headers, timeout=5)
        report_data.append({
            'Test': 'API Authorization',
            'Status': '✅ Passed' if api_response.status_code == 200 else '❌ Failed',
            'Details': f'Response: {api_response.status_code}'
        })
    except Exception as e:
        report_data.append({
            'Test': 'API Authorization',
            'Status': '❌ Failed',
            'Details': str(e)
        })
    
    # Test Workflow Listing
    try:
        workflows = get_workflows()
        report_data.append({
            'Test': 'Workflow Listing',
            'Status': '✅ Passed' if workflows else '❌ Failed',
            'Details': f'Found {len(workflows)} workflows'
        })
    except Exception as e:
        report_data.append({
            'Test': 'Workflow Listing',
            'Status': '❌ Failed',
            'Details': str(e)
        })
    
    # Display report
    report_df = pd.DataFrame(report_data)
    print("\nTest Report Summary:")
    print("===================")
    display(report_df)
    
    # Overall status
    all_passed = all(row['Status'].startswith('✅') for _, row in report_df.iterrows())
    print("\nOverall Status:", "✅ All Tests Passed" if all_passed else "❌ Some Tests Failed")
    
    return all_passed

# Generate test report
test_status = generate_test_report()